In [1]:
from transformers import pipeline
import json
from newspaper import Article
import time
import re
from urllib.parse import urlparse
import trafilatura

In [2]:
# Load small generative model for summarization and Q&A
flan_t5 = pipeline("text2text-generation", model="google/flan-t5-base")


In [3]:
# Load article URLs from the previous notebook
with open("resolved_articles.json", "r", encoding="utf-8") as f:
    resolved_articles = json.load(f)

print(f"✅ Loaded {len(resolved_articles)} resolved articles.")


✅ Loaded 10 resolved articles.


In [4]:
# Block clearly invalid or redirecting domains
def is_valid_news_domain(url):
    bad_domains = [
        "cloudflare.com", "facebook.com", "linkedin.com", "instagram.com",
        "forbes.com/sites", "subscribe.bloomberg.com", "t.co", "youtube.com"
    ]
    parsed = urlparse(url)
    full_domain = parsed.netloc + parsed.path
    return not any(bad in full_domain for bad in bad_domains)

# More refined relevance scoring
def is_content_relevant(text, title, min_words=100, keyword_match_threshold=0.2):
    if not text or len(text.split()) < min_words:
        return False
    if title:
        title_words = set(re.findall(r'\b\w+\b', title.lower()))
        text_words = set(re.findall(r'\b\w+\b', text.lower()))
        common = title_words & text_words
        score = len(common) / len(title_words) if title_words else 0
        return score >= keyword_match_threshold
    return True


In [5]:
#Main extraction function
def get_article_text(url, title_hint=None):
    try:
        if not is_valid_news_domain(url):
            print(f"🚫 Blocked domain: {url}")
            return "Blocked Domain", None

        title = title_hint or "Untitled Article"
        np_text = ""
        tf_text = ""

        # Try Newspaper3k
        try:
            article = Article(url)
            article.download()
            article.parse()
            title = article.title or title_hint or "Untitled Article"
            np_text = article.text.strip()
            if np_text:
                print(f"📄 Newspaper3k: {len(np_text.split())} words")
        except Exception as e:
            print(f"❌ Newspaper3k failed: {e}")

        # Try Trafilatura
        try:
            downloaded = trafilatura.fetch_url(url)
            if downloaded:
                tf_text = trafilatura.extract(downloaded) or ""
                if tf_text:
                    print(f"📄 Trafilatura: {len(tf_text.split())} words")
        except Exception as e:
            print(f"❌ Trafilatura failed: {e}")

        # Validate Newspaper3k first
        if np_text and is_content_relevant(np_text, title, min_words=100, keyword_match_threshold=0.2):
            return title, np_text

        # Accept Trafilatura if long enough (skip keyword match for fallback)
        if tf_text and len(tf_text.split()) > 150:
            return title, tf_text

        print(f"⚠️ Both tools returned poor content for: {url}")
        return title, None

    except Exception as e:
        print(f"❌ Extraction error for {url}:\n{e}")
        return "Failed to fetch title", None



In [6]:
def summarize_text_local(text, max_words=500):
    chunks = text.split('. ')
    summaries = []
    current_chunk = []

    for sentence in chunks:
        if len(" ".join(current_chunk + [sentence]).split()) <= max_words:
            current_chunk.append(sentence)
        else:
            chunk_text = ". ".join(current_chunk) + "."
            try:
                prompt = f"You are and expert news Analyst. Summarize the article below in **200 words**. Be factual and detailed.:\n\n{chunk_text}"
                response = flan_t5(prompt, max_new_tokens=400, min_new_tokens= 250, do_sample=False,
                    early_stopping=False)[0]['generated_text']
                summaries.append(response)
            except Exception as e:
                print(f"⚠️ Summarization error: {e}")
            current_chunk = [sentence]

    if current_chunk:
        try:
            chunk_text = ". ".join(current_chunk) + "."
            prompt = f"Summarize the following article:\n\n{chunk_text}"
            response = flan_t5(prompt, max_new_tokens=400, min_new_tokens= 250, do_sample=False,
                    early_stopping=False)[0]['generated_text']
            summaries.append(response)
        except Exception as e:
            print(f"⚠️ Summarization error: {e}")

    return " ".join(summaries)

In [7]:
summary_db = {}

for idx, article in enumerate(resolved_articles):
    print(f"\n🔍 [{idx+1}/{len(resolved_articles)}] {article['title']}")
    url = article["url"]

    title, full_text = get_article_text(url)
    if not full_text:
        summary_db[title] = {
            "topic": article["topic"],
            "url": url,
            "summary": "(No summary — article text not available.)",
            "full_text": None
        }
        continue

    summary = summarize_text_local(full_text)

    summary_db[title] = {
        "topic": article["topic"],
        "url": url,
        "summary": summary,
        "full_text": full_text
    }

    time.sleep(1)  # avoid rate-limiting



🔍 [1/10] Five Keys To Helping Traditional Businesses Embrace New Tech
❌ Newspaper3k failed: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/councils/forbestechcouncil/2025/06/25/five-keys-to-helping-traditional-businesses-embrace-new-tech/ on URL https://www.forbes.com/councils/forbestechcouncil/2025/06/25/five-keys-to-helping-traditional-businesses-embrace-new-tech/


Token indices sequence length is longer than the specified maximum sequence length for this model (672 > 512). Running this sequence through the model will result in indexing errors


📄 Trafilatura: 995 words

🔍 [2/10] Feds wasted millions on tech to detect fentanyl at the border, report finds
📄 Newspaper3k: 386 words
📄 Trafilatura: 420 words

🔍 [3/10] This Experimental Tech Allows Surgeons to See Through Blood
📄 Newspaper3k: 436 words
📄 Trafilatura: 436 words

🔍 [4/10] Texas Tech Shooting: What We Know
⚠️ Both tools returned poor content for: https://www.msn.com/en-us/news/crime/texas-tech-shooting-what-we-know/ar-AA1HkjAR

🔍 [5/10] Tech stocks power Nasdaq 100 to a record high as markets celebrate the Israel-Iran ceasefire
⚠️ Both tools returned poor content for: https://www.msn.com/en-us/money/markets/tech-stocks-power-nasdaq-100-to-a-record-high-as-markets-celebrate-the-israel-iran-ceasefire/ar-AA1HkLOI

🔍 [6/10] Inside Kelly Services Tech And AI Journey With CIO Sean Perry
🚫 Blocked domain: https://www.forbes.com/sites/peterhigh/2025/06/25/inside-kelly-services-tech-and-ai-journey-with-cio-sean-perry/

🔍 [7/10] Intel, Walmart, other tech companies reveal 500-pl

In [8]:
# Cleaned version for QnA
filtered_summary_db = {}

for title, data in summary_db.items():
    summary = data.get("summary", "")
    full_text = data.get("full_text", "")

    if summary and len(summary.split()) > 30:
        filtered_summary_db[title] = data
    elif full_text and len(full_text.split()) > 50:
        filtered_summary_db[title] = data
    else:
        print(f"⚠️ Skipping weak article: {title}")

print(f"\n✅ Filtered DB has {len(filtered_summary_db)} high-quality entries.")


⚠️ Skipping weak article: MSN
⚠️ Skipping weak article: Blocked Domain

✅ Filtered DB has 5 high-quality entries.


In [9]:
with open("summary_db.json", "w", encoding="utf-8") as f:
    json.dump(filtered_summary_db, f, indent=2, ensure_ascii=False)

print(f"\n✅ Summary DB saved with {len(filtered_summary_db)} entries.")



✅ Summary DB saved with 5 entries.


In [10]:
print("📚 Sample Titles in Summary DB:")
for i, title in enumerate(list(filtered_summary_db.keys())):
    print(f"{i+1}. {title}")


📚 Sample Titles in Summary DB:
1. Untitled Article
2. Feds wasted millions on tech to detect fentanyl at the border, report finds
3. This Experimental Tech Allows Surgeons to See Through Blood
4. Intel, Walmart, other tech companies reveal 500-plus Bay Area job cuts
5. Looking For Sustainable Tech? This New Repairable Phone May Be For You


In [11]:
# Load summary database
with open("summary_db.json", "r", encoding="utf-8") as f:
    summary_db = json.load(f)

# Preview full text of each article
for i, (title, entry) in enumerate(summary_db.items(), 1):
    print(f"\n{i}. 📰 Title: {title}")
    print(f"🔗 URL: {entry['url']}")
    
    summary = entry.get("summary", "")
    word_count = len(summary.split()) if summary else 0
    print(f"📝 Word count: {word_count}")
    
    if not summary:
        print("⚠️ Skipping — Summary missing.")
        continue
    
    # Show a short snippet for review
    print("📄 Summary Preview:\n", summary[:500], "...\n")
    print("-" * 80)


1. 📰 Title: Untitled Article
🔗 URL: https://www.forbes.com/councils/forbestechcouncil/2025/06/25/five-keys-to-helping-traditional-businesses-embrace-new-tech/
📝 Word count: 746
📄 Summary Preview:
 Revv CEO Adi Bathla is transforming auto repair with AI-powered tools for ADAS calibration, diagnostics and shop efficiency. At the intersection between businesses and technology is this irony: The industries that need technology the most are often the most resistant to adopting it. We’ve seen this firsthand at Revv, which serves the automotive repair industry: Many shops are still operating as they did decades ago. Even with all the technological revolution in the automotive industry, many shop ...

--------------------------------------------------------------------------------

2. 📰 Title: Feds wasted millions on tech to detect fentanyl at the border, report finds
🔗 URL: https://www.usatoday.com/story/news/politics/2025/06/25/feds-wasted-money-border-technology-fentanyl/84287735007/
📝 Wor